In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
data = pd.read_csv("../input/melbourne-housing-snapshot/melb_data.csv")
data.dropna(axis=0, subset=["Price"], inplace=True)

cols_to_use = ['Rooms', 'Distance', 'Landsize', 'BuildingArea', 'YearBuilt']
x = data[cols_to_use]
y = data.Price

In [10]:
categorical_cols = [col for col in x.columns if x[col].dtype == "object" and x[col].nunique() < 10]
numeric_cols = [col for col in x.columns if x[col].dtype in ["int64", "float64"]]

print(f"Numeric: {len(numeric_cols)} Categorical: {len(categorical_cols)}")

cols = categorical_cols + numeric_cols
x = x[cols]

In [11]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

numeric_transformer = Pipeline(steps=[
    ("impute", SimpleImputer(strategy="mean"))
])
categorical_transformer = Pipeline(steps=[
    ("impute", SimpleImputer(strategy="most_frequent")),
    ("oh", OneHotEncoder(handle_unknown="ignore"))
])
preprocessor = ColumnTransformer(transformers=[
    ("num", numeric_transformer, numeric_cols),
    ("cat", categorical_transformer, categorical_cols)
])

In [12]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=0)
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", model)
])

In [14]:
from sklearn.model_selection import cross_val_score

# Multiply by -1 since sklearn calculates *negative* MAE
# We set the number of folds with the cv parameter.
scores = -1 * cross_val_score(pipeline, x, y, cv=5, scoring="neg_mean_absolute_error")

print("MAE scores:\n", scores)
print("Average MAE score (across experiments):")
print(scores.mean())